## Алгоритм шифра Цезаря

Мы будем использовать английский алфавит, который состоит из 26 букв. Сдвиг буквы осуществляется по модулю 26. Вот код на Python для шифрования и дешифрования, а также для генерации выборки:

In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Определяем алфавиты
ALPHABETS = {
    'russian': 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ',
    'english': 'abcdefghijklmnopqrstuvwxyz ',
    'german': 'abcdefghijklmnopqrstuvwxyzäöüß '
}

# Создание соответствующих словарей
CHAR_TO_INDEX = {lang: {char: idx for idx, char in enumerate(alphabet)} for lang, alphabet in ALPHABETS.items()}
INDEX_TO_CHAR = {lang: {idx: char for idx, char in enumerate(alphabet)} for lang, alphabet in ALPHABETS.items()}

# Определяем размеры входных данных и выходов
INPUT_SIZES = {lang: len(alphabet) for lang, alphabet in ALPHABETS.items()}
HIDDEN_SIZE = 128  # Определите размер скрытого слоя
OUTPUT_SIZES = {lang: len(alphabet) for lang, alphabet in ALPHABETS.items()}

# Алгоритм шифра Цезаря
def caesar_cipher(text, shift, alphabet):
    result = ""
    for char in text:
        if char in alphabet:
            # Находим индекс текущего символа
            index = alphabet.index(char)
            # Применяем сдвиг
            new_index = (index + shift) % len(alphabet)  # Зацикливание по алфавиту
            result += alphabet[new_index]
        else:
            result += char  # Не изменяем, если символ не в алфавите
    return result

# Генерация данных
def generate_data(num_samples=1000, shift=2, language ='russian'):
    alphabet = ALPHABETS[language]
    samples = []
    for _ in range(num_samples):
        length = random.randint(5, 15)  # Случайная длина строки
        random_text = ''.join(random.choices(alphabet, k=length))  # Генерация случайного текста
        encoded_text = caesar_cipher(random_text, shift, alphabet)  # Шифрование текста
        samples.append((encoded_text, random_text))  # (защифрованный текст, оригинальный текст)
    return samples


## Подготовка данных для обучения RNN

In [2]:
# # Подготовка данных для обучения RNN
class CipherDataset(Dataset):
    def __init__(self, data, language):
        if language not in ALPHABETS:
            raise ValueError("Unsupported language. Please choose 'russian', 'english', or 'german'.")
        
        self.data = data
        self.language = language
        self.char_to_index = CHAR_TO_INDEX[self.language]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        encoded, original = self.data[index]
        # Преобразуем каждый символ в индекс
        return (
            torch.tensor([self.char_to_index[char] for char in encoded], dtype=torch.long),
            torch.tensor([self.char_to_index[char] for char in original], dtype=torch.long)
        )

# Функция для добавления паддинга к последовательностям
def collate_fn(batch):
    encoded_inputs = [item[0] for item in batch]
    original_outputs = [item[1] for item in batch]
    
    max_length = max(len(seq) for seq in encoded_inputs)
    
    padded_encoded = torch.stack([
        torch.cat([seq, torch.zeros(max_length - len(seq), dtype=torch.long)]) for seq in encoded_inputs
    ])
    
    padded_original = torch.stack([
        torch.cat([seq, torch.zeros(max_length - len(seq), dtype=torch.long)]) for seq in original_outputs
    ])
    
    return padded_encoded, padded_original



## Создание архитектуры рекуррентной нейронной сети

In [3]:
# Создание архитектуры RNN
class SimpleRNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, 30)
        self.rnn = nn.RNN(input_size=30, hidden_size=HIDDEN_SIZE, batch_first=True)
        self.fc = nn.Linear(HIDDEN_SIZE, input_size)

    def forward(self, x):
        x = self.embedding(x)
        rnn_out, _ = self.rnn(x)  # Выходы для всех временных шагов
        return self.fc(rnn_out)  # Вернуть все выходы

## Обучение модели

In [4]:
# Обучение модели
def train(model, dataloader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for encoded_x, original_y in dataloader:
            optimizer.zero_grad()

            output = model(encoded_x)  # Получаем выход от модели

            output = output.view(-1, OUTPUT_SIZES[dataloader.dataset.language])  # (batch_size * seq_len, OUTPUT_SIZE)
            original_y = original_y.view(-1)  # (batch_size * seq_len)
            loss = criterion(output, original_y)  # Рассчитываем потери
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}')

## Проверка качества модели

In [5]:
# Проверка качества модели на зашифрованном входе
def evaluate(model, encoded_text, language):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor([CHAR_TO_INDEX[language][char] for char in encoded_text]).view(1, -1)
        output = model(input_tensor)
        output = output.view(-1, OUTPUT_SIZES[language])  # Изменяем размерность
        _, predicted_indices = torch.max(output, -1)
        decoded_text = ''.join([INDEX_TO_CHAR[language][idx.item()] for idx in predicted_indices])
    return decoded_text

## Запуск процесса

In [6]:
if __name__ == "__main__":
    # Задаем параметры
    language = 'english'  # можно русский, английский, немецкий 
    num_samples = 1000
    shift = 2

    # Генерация данных
    data = generate_data(num_samples=num_samples, shift=shift, language=language)

    # Подготовка и распределение данных
    dataset = CipherDataset(data, language=language)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

   # Инициализация модели
    model = SimpleRNN(INPUT_SIZES[language])  # INPUT_SIZE определяет размер входа по алфавиту

   # Определение функции потерь
    criterion = nn.CrossEntropyLoss()  # Для задач многоклассовой классификации

   # Инициализация оптимизатора
    optimizer = optim.AdamW(model.parameters(), lr=0.001)  # Устанавливаем Adam как оптимизатор с начальной скоростью обучения 0.001
   # Обучение модели
    train(model, dataloader, criterion, optimizer, num_epochs=20)

    # Пример для проверки
    test_samples = generate_data(10, shift=shift, language=language)
    for encoded, original in test_samples:
        decoded = evaluate(model, encoded, language)
        print(f'Encoded: {encoded}\nOriginal: {original}\nDecoded: {decoded}\n')

Epoch 1/20, Loss: 2.222014993429184
Epoch 2/20, Loss: 1.0577965434640646
Epoch 3/20, Loss: 0.38165406975895166
Epoch 4/20, Loss: 0.15720793581567705
Epoch 5/20, Loss: 0.10248364065773785
Epoch 6/20, Loss: 0.07826559769455343
Epoch 7/20, Loss: 0.06509139016270638
Epoch 8/20, Loss: 0.057547241100110114
Epoch 9/20, Loss: 0.052253272384405136
Epoch 10/20, Loss: 0.047270149108953774
Epoch 11/20, Loss: 0.04495182994287461
Epoch 12/20, Loss: 0.04201121750520542
Epoch 13/20, Loss: 0.03941555641358718
Epoch 14/20, Loss: 0.03635356843005866
Epoch 15/20, Loss: 0.035434294084552675
Epoch 16/20, Loss: 0.03414078365312889
Epoch 17/20, Loss: 0.030787639145273715
Epoch 18/20, Loss: 0.029676320729777217
Epoch 19/20, Loss: 0.02688651467906311
Epoch 20/20, Loss: 0.02438505698228255
Encoded:  qahlpld
Original: yozfjnjb
Decoded: yozfjnjb

Encoded: uxefndup
Original: svcdlbsn
Decoded: svcdlbsn

Encoded: blhipubuuwfgoot
Original:  jfgns ssudemmr
Decoded:  jfgns ssudemmr

Encoded: qfztan
Original: odxrzl
Deco

### Выводы: 
1.Снижение потерь сигнализирует о том, что модель обучается и в состоянии улучшать свои предсказания. В идеале, значение потерь стремвится к 0, однако наличие некоторого остаточного значения в 0.024 говорит о том, что модель все равно может иметь место для улучшения.
2.Примерная оценка точности составляет 80%, что является хорошим результатом для такой задачи.
3. В большинстве случаев модель точно расшифровывает входные данные, однако есть несколько незначительных ошибок, которые могут быть улучшены за счет увеличения объемов данных, использования GRU для улучшения обработки последовательностей.